In [6]:
page_size_options = {
    "Letter" : [21.6, 27.9],
    "Legal" : [21.6, 35.6],
    "Tabloid" : [27.9, 43.2],
    "A4": [21.0, 29.7]
}

In [30]:
from fpdf import FPDF # needs fpdf2
from PIL import Image
import requests
from io import BytesIO

pdf_export_name = "export.pdf"

def print_pdf(image_url, cut_sz, cutouts, margin:int, page_size:str, orientation:str="L", unit:str="cm") -> None:
    pdf = FPDF(orientation=orientation, unit=unit, format=page_size)
    
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    
    scale = [a/b for a,b in zip(img.size, cut_sz)]
    scale.extend(scale)
    
    for pg in cutouts:
        pdf.add_page()
        if pg[2] > cut_sz[0] or pg[3] > cut_sz[1] or pg[0] < 0 or pg[1] < 0:
            # handle the overflow page issue
            pg_in_bounds = [max(0,pg[0]), max(0,pg[1]), min(cut_sz[0],pg[2]), min(cut_sz[1],pg[3])]
            clip = img.crop([round(a*b) for a,b in zip(pg_in_bounds,scale)])
        else:
            clip = img.crop([round(a*b) for a,b in zip(pg,scale)])
        
        with BytesIO() as im_cache:
            clip.save(im_cache, format="PNG")
            pdf.image(im_cache, margin, margin, 
                      (page_size[0]-margin*2) * min((cut_sz[0]-pg[0])/(pg[2]-pg[0]),1), 
                      (page_size[1]-margin*2) * min((cut_sz[1]-pg[1])/(pg[3]-pg[1]),1))
        
    pdf.output(pdf_export_name, "F")

In [31]:
import asyncio

class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
            timer.start()
        return debounced
    return decorator

In [32]:
from typing import List

objects_to_draw = []
class Drawing_obj():
    def __init__(self, x,y, width=5, height=5, color='#000000'):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.color = color
        self.selected = False
        objects_to_draw.append(self)
        
    def set_x_y(self,x_in,y_in)  :
        self.x = x_in
        self.y = y_in
    
    def draw(self):
        pass
    
    def is_selected(self,x_in, y_in):
        return False
    
    def set_selected(self,state):
        self.selected = state
                           
            
class Reference_Plus_obj(Drawing_obj):
    def draw(self):
        if self.selected:
            m_canvas[1].stroke_style = '#FF0000'
        else:
            m_canvas[1].stroke_style = self.color
        m_canvas[1].stroke_line(self.x, self.y - (self.height*0.5),
                           self.x, self.y + (self.height*0.5))
        m_canvas[1].stroke_line(self.x - (self.width*0.5), self.y,
                           self.x + (self.width*0.5), self.y)
        
    def is_selected(self,x_in, y_in):
        x_coord = self.x - (self.width*0.5)
        y_coord = self.y - (self.height*0.5)

        if x_in > x_coord and x_in < (x_coord+ self.width) and  y_in > y_coord  and y_in < (y_coord  + self.height):
            self.set_selected(True)
            return True
        else:
            self.set_selected(False)
            return False
                           
                           
class Page_obj(Drawing_obj):
    def __init__(self, x:int,y:int, width:int=5, height:int=5, color:str='#000000', margin:List[int]=[0,0]):
        self.margin = margin
        super(Page_obj, self).__init__(x, y, width, height, color)
        
    def draw(self):
        if self.selected:
            m_canvas[1].stroke_style = "#440044"
        else:
            m_canvas[1].stroke_style = self.color
        m_canvas[1].stroke_rect(self.x, self.y, self.width, self.height)
        m_canvas[1].stroke_style = "#440000"
        m_canvas[1].stroke_rect(self.x + self.margin[0], self.y + self.margin[1], 
                self.width - 2*self.margin[0], self.height - 2*self.margin[1])
        
    def is_selected(self,x_in, y_in):
        # within ten percent of the box boundary
        x_diff = x_in - self.x
        y_diff = y_in - self.y

        if ((abs(x_diff) < self.width*0.05 and y_diff < self.height*1.05 and y_diff > self.height*-0.05) or
            (abs(x_diff - self.width) < self.width*0.05 and y_diff < self.height*1.05 and y_diff > self.height*-0.05) or
            (abs(y_diff) < self.height*0.05 and x_diff < self.width*1.05 and x_diff > self.width*-0.05) or
            (abs(y_diff - self.height) < self.height*0.05 and x_diff < self.width*1.05 and x_diff > self.width*-0.05)):
            self.set_selected(True)
            return True
        else:
            self.set_selected(False)
            return False
        
    def get_printable_area(self) -> List[int]:
        '''
        Returns Left, Top, Right, and Bottom extents of the page's printable area
        '''
        return [self.x + self.margin[0], self.y + self.margin[1], 
                self.x + self.width - self.margin[0], self.y + self.height - self.margin[1]]


In [36]:
import ipywidgets as widgets
from ipycanvas import Canvas, MultiCanvas
from ipyevents import Event
from IPython.display import FileLink, FileLinks
import validators
from PIL import Image
import requests
from io import BytesIO

canvas_x = 900 # need to get it to fit these to the layout stretch? It tries to keep the aspect ratio
canvas_y = 300

scale_x = 1 # scale in px/cm
scale_y = 1 # scale in px/cm

x_color = '#0000FF'
y_color = '#00FF00'

move_count = 0

def tile_pages(bounds):
    '''
    @param bounds [[x1,y1],[x2,y2]] extents of the tiling area
    '''
    pg_size = page_size_options[page_size_select.value]
    pg_size = scale_x_y(pg_size)
    pg_margin = margin.value
    pg_margin = scale_x_y([pg_margin, pg_margin])
    pg_overlap = [overlap.value / 100 / 2 * a + b for a,b in zip(pg_size, pg_margin)] #already in px
    
    track_extent_init = [ bound - margin for bound, margin in zip(bounds[0], pg_margin)]
    track_extent = track_extent_init.copy()
    track_max = [ bound - margin for bound, margin in zip(bounds[1], pg_margin)]
    while track_extent[1] < track_max[1]:         # inside y bounds (partial page overflow ok)
        track_extent[0] = track_extent_init[0]
        while track_extent[0] < track_max[0]:     # inside x bounds (partial page overflow ok)
            # TODO: some kind of check to see if the page set to be drawn is empty
            obj = Page_obj(round(track_extent[0]), round(track_extent[1]), pg_size[0], pg_size[1], '#444444',
                           margin=pg_margin)
            track_extent[0] += pg_size[0] - 2 * pg_overlap[0]
            
        track_extent[1] += pg_size[1] - 2 * pg_overlap[1]
            
            
def scale_x_y(in_list):
    assert len(in_list) == 2
    return [round(a*b) for a,b in zip(in_list, [scale_x, scale_y])]
            
    
def set_scale():
    global scale_x
    global scale_y
    
    x_ref_abs = line_1_dim.value
    y_ref_abs = line_2_dim.value
    
    x_ref_pix = abs(x_ref[0].x - x_ref[1].x)
    y_ref_pix = abs(y_ref[0].y - y_ref[1].y)
    
    if (x_ref_abs <= 0 or y_ref_abs <= 0):
        print("Invalid reference")
    else:
        scale_x = x_ref_pix / x_ref_abs
        scale_y = y_ref_pix / y_ref_abs
        
        
def clear_pages():
    global objects_to_draw
    tmp_objects = []
    for obj in objects_to_draw:
        if not isinstance(obj, Page_obj):
            tmp_objects.append(obj)
            
    objects_to_draw = tmp_objects
    
    
def delete_selected_page():
    global objects_to_draw
    tmp_objects = []
    for obj in objects_to_draw:
        if not isinstance(obj, Page_obj) or not obj.selected:
            tmp_objects.append(obj)
    objects_to_draw = tmp_objects
            
    
def enforce_margin_limit():
    min_pg_dim = min(page_size_options[page_size_select.value])
    if margin.value > min_pg_dim / 2:
        margin.value = 0
    

def update(change = None):
    canvas_restart()
    [o.draw() for o in objects_to_draw]
    
    
def autoarrange_pages(change = None):
    clear_pages()
    image_bounds = [[0,0],[canvas_x, canvas_y]]
    set_scale()
    tile_pages(image_bounds)
    update()
    
    
def print_all_pages():
    pgs = []
    for obj in objects_to_draw:
        if isinstance(obj, Page_obj):
            pgs.append(obj.get_printable_area())
    print_pdf(image_url.value, 
              [canvas_x, canvas_y], 
              pgs, 
              margin.value, 
              page_size=page_size_options[page_size_select.value],
              orientation="P")
    
    
def read_img(url:str):
    try:
        response = requests.get(url)
        disp.value = BytesIO(response.content).getvalue()
    except:
        print("Invalid image url")
        
        
def canvas_restart():
    m_canvas[1].clear()
    

def handle_mouse_down(x, y):
    if [o for o in objects_to_draw if o.selected]:
        [o.set_selected(False) for o in objects_to_draw if o.selected]
        return False
    [check_region.is_selected(x,y) for check_region in objects_to_draw]
    update()
    return True
    

@debounce(0.05)
def handle_mouse_move(x, y):
    if [o for o in objects_to_draw if o.selected]:
        selected_item = [o for o in objects_to_draw if o.selected][-1]
        selected_item.set_x_y(x,y)
        update()
        
def move_selected(x, y):
    if [o for o in objects_to_draw if o.selected]:
        selected_item = [o for o in objects_to_draw if o.selected][-1]
        x_start = selected_item.x
        y_start = selected_item.y
        selected_item.set_x_y(x_start + x, y_start + y)


style = {'description_width': 'initial'}

'''
Image X-dimension reference widget
'''
line_1_dim = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=1000.0,
    step=0.1,
    description=f"<b><font color='{x_color}'>X reference dimension (cm):</b",
    disabled=False,
    layout=widgets.Layout(width='auto', grid_area='line_1_dim'),
    style=style
)
line_1_dim.observe(update)
    
'''
Image Y-dimension reference widget
'''
line_2_dim = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=1000.0,
    step=0.1,
    description=f"<b><font color='{y_color}'>Y reference dimension (cm):</b>",
    disabled=False,
    layout=widgets.Layout(width='auto', grid_area='line_2_dim'),
    style=style
)
line_2_dim.observe(update)

'''
Page size dropdown widget
'''
page_size_select = widgets.Dropdown(
    options=page_size_options.keys(),
    value=list(page_size_options.keys())[0],
    description='Page size: ',
    disabled=False,
    layout=widgets.Layout(width='auto', grid_area='page_size'),
    style=style
)
def margin_eventhandler(change):
    enforce_margin_limit()
    update()
page_size_select.observe(margin_eventhandler)

'''
Overlap percent float widget
'''
overlap = widgets.BoundedFloatText(
    value=5,
    min=0,
    max=100,
    step=0.1,
    description='Tiling Overlap (%):',
    disabled=False,
    layout=widgets.Layout(width='auto', grid_area='overlap'),
    style=style
)
overlap.observe(update)

'''
Margin size float widget
'''
margin = widgets.BoundedFloatText(
    value=0,
    min=0,
    max=100,
    step=0.1,
    description='Page Margin (cm):',
    disabled=False,
    layout=widgets.Layout(width='auto', grid_area='margin'),
    style=style
)
margin.observe(margin_eventhandler)

'''
Button widget for recalculating pages
'''
recalc = widgets.Button(
    disabled=False,
    description='Recalculate Pages',
    layout=widgets.Layout(width='auto', grid_area='recalculate'),
    style=style)
recalc.on_click(autoarrange_pages)

'''
Button widget for clearing pages
'''
clear_pgs = widgets.Button(
    disabled=False,
    description='Clear Pages',
    layout=widgets.Layout(width='auto', grid_area='clear'),
    style=style)
def clear_pgs_eventhandler(change):
    clear_pages()
    update()
clear_pgs.on_click(clear_pgs_eventhandler)


'''
Button widget for printing pages to pdf
'''
page_print = widgets.Button(
    disabled=False,
    description='Print to PDF',
    layout=widgets.Layout(width='auto', grid_area='print'),
    style=style
)
file_links_shown = []
def print_pages_eventhandler(change):
    if validators.url(image_url.value):
        print_all_pages()
        if download_link not in file_links_shown:
            display(download_link)
            file_links_shown.append(download_link)
page_print.on_click(print_pages_eventhandler)

'''
Image URL widget
'''
image_url = widgets.Text(
    value=None,
    placeholder='Enter image URL',
    description='Image:',
    disabled=False,
    layout=widgets.Layout(width='auto', grid_area='image_url'),
    style=style
)
def img_url_dim_eventhandler(change):
    if change.new is not None and len(change.new) > 0: 
        read_img(change.new)
        m_canvas[0].clear()
        m_canvas[0].draw_image(disp, 0, 0, canvas_x, canvas_y)
        update()
image_url.observe(img_url_dim_eventhandler, names="value")

'''
Image display widget
'''
disp = widgets.Image(
    layout=widgets.Layout(width='auto', grid_area='display')
)

'''
Download link
'''
download_link = FileLink(pdf_export_name, result_html_prefix="Download PDF: ")

'''
Canvas workspace/drawing widget
'''
m_canvas = MultiCanvas(
    2,
    width = canvas_x,
    height = canvas_y,
    layout=widgets.Layout(width='auto', grid_area='display')
)
m_canvas[1].on_mouse_down(handle_mouse_down)
m_canvas[1].on_mouse_move(handle_mouse_move)
canvas_restart()

'''
Arrange UI/widget layout
'''
ui = widgets.GridBox(
    children=[image_url, line_1_dim, line_2_dim, recalc, page_print,
             m_canvas, margin, overlap, page_size_select, clear_pgs],
    layout=widgets.Layout(
    width='100%',
    grid_template_columns='25% 25% 25% 25%',
    grid_template_areas='''
    "image_url image_url image_url image_url"
    "line_1_dim line_2_dim overlap margin"
    "page_size clear recalculate print"
    "display display display display"
    ''')
)

hotkeys = Event(source=m_canvas, watched_events=['keyup'])
def handle_event(event):
    if (event["key"] == "Delete"):
        delete_selected_page()
        update()
    if (event["key"] == "w"):
        move_selected(0,-5)
        update()
    if (event["key"] == "s"):
        move_selected(0,5)
        update()
    if (event["key"] == "a"):
        move_selected(-5,0)
        update()
    if (event["key"] == "d"):
        move_selected(5,0)
        update()
    if (event["key"] == "Enter"):
        [o.set_selected(False) for o in objects_to_draw if o.selected]
        update()
hotkeys.on_dom_event(handle_event)
     
objects_to_draw = []
# Drop in X reference pluses
x_ref_1 = Reference_Plus_obj(0, canvas_y*0.5, 15, 15, x_color)
x_ref_2 = Reference_Plus_obj(canvas_x, canvas_y*0.5, 15, 15, x_color)
x_ref = [x_ref_1, x_ref_2]
# Drop in Y reference pluses
y_ref_1 = Reference_Plus_obj(canvas_x*0.5, 0, 15, 15, y_color)
y_ref_2 = Reference_Plus_obj(canvas_x*0.5, canvas_y, 15, 15, y_color)
y_ref = [y_ref_1, y_ref_2]
# TODO Add a drawing to label scale that updates based on reference vals


display(ui)
update()

GridBox(children=(Text(value='', description='Image:', layout=Layout(grid_area='image_url', width='auto'), pla…

C:\Users\Bowen Shaner\Documents\PrintTile\export.pdf